In [1]:
# ================================
# Step 1: Import Libraries
# ================================
!pip install tensorflow

import zipfile
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
# ================================
# Step 2: Mount Google Drive
# ================================
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Path to your ZIP file in Drive
zip_path = "/content/drive/MyDrive/kindey_stone_dataset.zip"

In [4]:
# Extract it into /content/dataset
extract_path = "/content/dataset"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [5]:
# Paths
train_dir = os.path.join(extract_path, "train")
val_dir   = os.path.join(extract_path, "val")

In [6]:
# ================================
# Step 3: Image Generators
# ================================
train_dir = "/content/dataset/kindey_stone_dataset/train"
val_dir   = "/content/dataset/kindey_stone_dataset/val"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)


Found 35457 images belonging to 2 classes.
Found 3364 images belonging to 2 classes.


In [7]:
# ================================
# Step 4: Load VGG19
# ================================
base_model = VGG19(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False  # Freeze base model

x = Flatten()(base_model.output)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(train_gen.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=x)

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
# ================================
# Step 5: Compile
# ================================
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [9]:
# ================================
# Step 6: Train
# ================================
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1109/1109 ━━━━━━━━━━━━━━━━━━━━ 581s 509ms/step - accuracy: 0.6531 - loss: 0.6425 - val_accuracy: 0.8701 - val_loss: 0.3555
Epoch 2/10
1109/1109 ━━━━━━━━━━━━━━━━━━━━ 583s 487ms/step - accuracy: 0.7826 - loss: 0.4618 - val_accuracy: 0.9037 - val_loss: 0.2681
Epoch 3/10
1109/1109 ━━━━━━━━━━━━━━━━━━━━ 535s 483ms/step - accuracy: 0.8160 - loss: 0.4061 - val_accuracy: 0.8948 - val_loss: 0.2756
Epoch 4/10
1109/1109 ━━━━━━━━━━━━━━━━━━━━ 533s 480ms/step - accuracy: 0.8411 - loss: 0.3687 - val_accuracy: 0.9108 - val_loss: 0.2286
Epoch 5/10
1109/1109 ━━━━━━━━━━━━━━━━━━━━ 536s 484ms/step - accuracy: 0.8521 - loss: 0.3390 - val_accuracy: 0.9227 - val_loss: 0.1985
Epoch 6/10
1109/1109 ━━━━━━━━━━━━━━━━━━━━ 530s 478ms/step - accuracy: 0.8580 - loss: 0.3230 - val_accuracy: 0.9370 - val_loss: 0.1560
Epoch 7/10
1109/1109 ━━━━━━━━━━━━━━━━━━━━ 533s 481ms/step - accuracy: 0.8696 - loss: 0.3028 - val_accuracy: 0.9468 - val_loss: 0.1334
Epoch 8/10
1109/1109 ━━━━━━━━━━━━━━━━━━━━ 535s 482ms/step - ac

In [10]:
# ================================
# Step 7: Save Model
# ================================
model.save("/content/vgg19_kidney_stone.h5")
print("✅ Model saved successfully at /content/vgg19_kidney_stone.h5")

✅ Model saved successfully at /content/vgg19_kidney_stone.h5
